In [22]:
import boto3
import tempfile

import botocore.utils as boto_utils

TEST_TEXT = ('Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do '
             'eiusmod tempor incididunt ut labore et dolore magna aliqua.')

def boto3_session(access_key, secret_key):

    return boto3.Session(
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key)



  

In [23]:
def boto3_s3_client(access_key, secret_key, endpoint_url):

    # Create a session and then a client from it.
    session = boto3_session(access_key, secret_key)
    client = session.client(
        's3',
        region_name='iggy-1',
        use_ssl=False,
        verify=False,
        endpoint_url=endpoint_url,
        config=boto3.session.Config(
            signature_version='s3',
            s3={
                'addressing_style': 'path'
            }
        ))

    # This minor workaround is required to avoid having boto3 attempt to fix
    # a given endpoint_url using the AWS DNS pattern.
    client.meta.events.unregister('before-sign.s3', boto_utils.fix_s3_host)

    # All finished here. We can start using the client as expected now
    return client

In [24]:
def list_buckets(client):

    # See the following URL for more details about this call:
    # http://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Client.list_buckets
    lsb_resp = client.list_buckets()

    for bucket in lsb_resp['Buckets']:
        yield bucket['Name']

In [25]:

def list_objects(client, bucket):
    """
    Return a generator that iterates through the keys contained within the
    specified bucket.
    """

    # See the following URL for more details about this call:
    # http://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Client.list_objects
    lsb_resp = client.list_objects(
        Bucket=bucket)

    for obj in lsb_resp['Contents']:
        yield obj['Key']

In [26]:
def put_object(client, bucket, key, fd):

    # See the following URL for more details about this call:
    # http://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Client.put_object
    put_resp = client.put_object(
        Body=fd,
        Bucket=bucket,
        Key=key
    )

    # Return the object's version
    return put_resp['VersionId']

In [27]:
def get_object(client, bucket, key):

    # See the following URL for more details about this call:
    # http://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Client.get_object
    get_resp = client.get_object(
        Bucket=bucket,
        Key=key)

    return get_resp['Body']

In [28]:
def delete_object(client, bucket, key):

    # See the following URL for more details about this call:
    # http://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Client.delete_object
    get_resp = client.delete_object(
        Bucket=bucket,
        Key=key)

    return get_resp['VersionId']

In [29]:
def run_example(bucket, access_key, secret_key, endpoint_url):

    # Create a new client to play with
    client = boto3_s3_client(access_key, secret_key, endpoint_url)

    # List the buckets to see what's in the root context
    print('Listing buckets...')
    for b in list_buckets(client):
        print(b)

    # Let's generate something to put into a bucket
    fd = tempfile.TemporaryFile()
    fd.write(TEST_TEXT)

    # Make sure the file is flushed and then seek to the start for reading
    fd.flush()
    fd.seek(0)

    # Put the object - passing a file object here is okay
    version_1 = put_object(client, bucket, 'test', fd)

    # Clean up the temp file
    fd.close()

    # Put the object again - passing a string or bytearray also works for this call
    version_2 = put_object(client, bucket, 'test', TEST_TEXT)

    print('Version started at: {}. Latest object version: {}.'.format(
        version_1, version_2))

    # Get the object
    object_fd = get_object(client, bucket, 'test')
    object_content = object_fd.read()

    print('Body of object: "{}"'.format(
        object_content))

    print('Put content matches retrieved content: {}'.format(
        object_content == TEST_TEXT))

    # List the objects in the bucket now
    print('Listing objects...')
    for key in list_objects(client, bucket):
        print(key)

    # Delete the object we put
    del_version = delete_object(client, bucket, 'test')

    print('Deleted object. Version is now: {}'.format(del_version))


In [30]:
# Run the example!
if __name__ == '__main__':
    run_example(
        bucket='',
        access_key='',
        secret_key='',
        endpoint_url='')

ValueError: Invalid endpoint: 